In [1]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999;

<IPython.core.display.Javascript object>

In [6]:
import sys
sys.path.append('../../')

from matplotlib import pyplot
import numpy as np
import pandas
from matplotlib import pyplot as plt
from scipy.stats import rankdata
from IPython.display import display, HTML

from bayesian_benchmarks.database_utils import Database
from bayesian_benchmarks.data import classification_datasets, _ALL_REGRESSION_DATATSETS, _ALL_CLASSIFICATION_DATATSETS
ALL_DATATSETS = {}
ALL_DATATSETS.update(_ALL_REGRESSION_DATATSETS)
ALL_DATATSETS.update(_ALL_CLASSIFICATION_DATATSETS)
from bayesian_benchmarks.data import regression_datasets


In [3]:
def rankarray(A):
    ranks = []
    for a in A:
        ranks.append(rankdata(a))
    return np.array(ranks)


def read_regression_classification(fs, models_names, datasets, task):
    if task == 'classification':
        fields = ['dataset', 'N', 'D', 'K'] + [m[1] for m in models_names]
    else:
        fields = ['dataset', 'N', 'D'] + [m[1] for m in models_names]

    results = {}
    for f in fs:
        results[f] = {'table':{f:[] for f in fields}, 'vals':[]}

    with Database('../tasks/elevators_temp.db') as db:

        for dataset in datasets:
            for f in fs:
                results[f]['table']['dataset'].append(dataset[:10])
                results[f]['table']['N'].append(ALL_DATATSETS[dataset].N)
                results[f]['table']['D'].append(ALL_DATATSETS[dataset].D)
                if task == 'classification':
                    results[f]['table']['K'].append(ALL_DATATSETS[dataset].K)

            row = {f:[] for f in fs}
            for model, name in models_names:
                res = db.read(task, fs, {'model':model, 
                                         'dataset':dataset})
                    
                if len(res) == 0:
                    for f in fs:
                        results[f]['table'][name].append('')
                        row[f].append(np.nan)
                else:
                    print('{} {} {}'.format(model, dataset, len(res)))
                    for i, f in enumerate(fs):
                        L = [float(l[i]) for l in res]
                        m = np.average(L)
                        std = np.std(L) if len(L) > 1 else np.nan
                        if m < 1000 and m > -1000:
                            r = '{:.3f}({:.3f})'.format(m, std)
                            row[f].append(m)
                        else:
                            r = 'nan'
                            row[f].append(np.nan)

                        results[f]['table'][name].append(r)

            #             stderr = np.std(L)/float(len(L))**0.5
            #             r = '{:.3f} ({:.3f})'.format(m, stderr)
            for f in fs:   
                results[f]['vals'].append(row[f])


    for f in fs:
        if 'unnormalized' not in f:
            vals = np.array(results[f]['vals'])

            avgs = np.nanmean(vals, 0)
            meds = np.nanmedian(vals, 0)
            rks = np.nanmean(rankarray(vals), 0)

            for s, n in [[avgs, 'avg'], [meds, 'median'], [rks, 'avg rank']]:
                results[f]['table']['dataset'].append(n)
                results[f]['table']['N'].append('')
                results[f]['table']['D'].append('')
                if task == 'classification':
                    results[f]['table']['K'].append('')
                for ss, name in zip(s, [m[1] for m in models_names]):
                    results[f]['table'][name].append('{:.3f}'.format(ss))
    
    return results, fields


In [4]:
models_names = [['RegNet', 'SGD'], ['RegNetpcaess', 'PCA+ESS'], ['RegNetpcavi', 'PCA+VI'], 
               ['RegNetpcalow_rank_gaussian', 'PCA+SWAG']]
regression_datasets = ['wilson_elevators', 'wilson_keggdirected', 'wilson_keggundirected', 'wilson_protein']
fs = 'test_loglik', 'test_rmse', 'test_loglik_unnormalized', 'test_rmse_unnormalized', 'test_calibration'

results, fields = read_regression_classification(fs, models_names, regression_datasets, 'regression')


OperationalError: no such table: regression

In [ ]:
print('normalised test loglikelihood')
display(HTML(pandas.DataFrame(results['test_loglik']['table'], columns=fields).to_html(index=False)))
# print(pandas.DataFrame(results['test_loglik']['table'], columns=fields).to_latex())

# print('unnormalized test loglikelihood')
# display(HTML(pandas.DataFrame(results['test_loglik_unnormalized']['table'], columns=fields).to_html(index=False)))


# print('normalised test rmse')
# display(HTML(pandas.DataFrame(results['test_rmse']['table'], columns=fields).to_html(index=False)))

print('normalised test rmse')
display(HTML(pandas.DataFrame(results['test_rmse_unnormalized']['table'], columns=fields).to_html(index=False)))

print('test_calibration')
display(HTML(pandas.DataFrame(results['test_calibration']['table'], columns=fields).to_html(index=False)))

In [ ]:
fs = ['test_loglik', 'test_acc']
results, fields = read_regression_classification(fs, models_names, classification_datasets, 'classification')


In [ ]:
print('test loglikelihood')
display(HTML(pandas.DataFrame(results['test_loglik']['table'], columns=fields).to_html(index=False)))
# print(pandas.DataFrame(results['test_loglik']['table'], columns=fields).to_latex())

print('test accuracy')
display(HTML(pandas.DataFrame(results['test_acc']['table'], columns=fields).to_html(index=False)))



In [ ]:
# fields = ['dataset', 'N', 'D']

                
# colours = ['C{}'.format(i) for i in range(10)]

# fields = fields + [m[1] for m in models_names]
# results = {f:[] for f in fields}


# for dataset in regression_datasets:
    
#     fig, axs = plt.subplots(1, 2, figsize=(10, 5))

#     results['dataset'].append(dataset)
#     results['N'].append(ALL_REGRESSION_DATATSETS[dataset].N)
#     results['D'].append(ALL_REGRESSION_DATATSETS[dataset].D)

#     for (model, name), c in zip(models_names, colours):
#         with Database('../results/results.db') as db:
#             d = {'model':model, 'dataset':dataset}

#             res = db.read('active_learning_continuous', ['total_loglik', 'total_rmse'], d) 
#         if len(res)>0:
#             test_ll = res[0][0]
#             test_acc = res[0][1]

#             axs[0].plot(test_ll, label=model, color=c)r
#             axs[1].plot(test_acc, label=model, color=c)
#     axs[0].set_ylim(-10, 10)
#     plt.title('{} {} {}'.format(dataset,
#                                    ALL_REGRESSION_DATATSETS[dataset].N,
#                                    ALL_REGRESSION_DATATSETS[dataset].D))
#     plt.legend()
#     plt.show()


In [ ]:

# fields = ['dataset', 'N', 'D', 'K']

# models_names = [['linear', 'lin'],
#                 ['variationally_sparse_gp', 'SVGP'],
#                 ['deep_gp_doubly_stochastic','DGP'],
#                 ['svm', 'svm'],
#                 ['knn', 'knn'],
#                 ['naive_bayes', 'nb'],
#                 ['decision_tree', 'dt'],
#                 ['random_forest', 'rf'],
#                 ['gradient_boosting_machine', 'gbm'],
#                 ['adaboost', 'ab'],
#                 ['mlp', 'mlp'],
#                 ]
                
# colours = ['C{}'.format(i) for i in range(10)]

# fields = fields + [m[1] for m in models_names]
# results = {f:[] for f in fields}


# for dataset in classification_datasets[:4]:  # don't show them all...
    
#     fig, axs = plt.subplots(1, 2, figsize=(10, 5))

#     results['dataset'].append(dataset)
#     results['N'].append(ALL_CLASSIFICATION_DATATSETS[dataset].N)
#     results['D'].append(ALL_CLASSIFICATION_DATATSETS[dataset].D)
#     results['K'].append(ALL_CLASSIFICATION_DATATSETS[dataset].K)

#     for (model, name), c in zip(models_names, colours):
#         with Database('../results/results.db') as db:
#             d = {'model':model, 'dataset':dataset}

#             res = db.read('active_learning_discrete', ['test_loglik', 'total_acc'], d) 
#         if len(res)>0:
#             test_ll = res[0][0]
#             test_acc = res[0][1]

#             axs[0].plot(test_ll, label=model, color=c)
#             axs[1].plot(test_acc, label=model, color=c)

#     plt.title('{} {} {} {}'.format(dataset,
#                                    ALL_CLASSIFICATION_DATATSETS[dataset].N,
#                                    ALL_CLASSIFICATION_DATATSETS[dataset].D,
#                                    ALL_CLASSIFICATION_DATATSETS[dataset].K))
#     plt.legend()
#     plt.show()